# 工作台管理

In [37]:
a = [1,2,3,4]
b = a
b.clear()
a

[]

In [36]:
h.append([{"role": "user", "content": "hello"}], call_at="today", return_at="today")
h.get()

{'messages': [{'role': 'user', 'content': 'hello'}],
 'call_at': 'today',
 'return_at': 'today'}

In [35]:
h.get(0)

{'messages': [{'role': 'user', 'content': 'hi'}]}

**任务树：**
- 【0级】对话 / 知识 / 数据 等
- 【1级 `write` 写作】对话 / 知识 / 数据 等
- 【2级 `from_outline` 扩写】对话 / 知识 / 数据 等

## 新任务

`write`/`from_outline` 执行时都将产生新任务。

## 新对话

新任务将同时产生新对话。<br>
另外，可以使用 `new_chat` 直接生成新对话。

**0级新对话**

In [12]:
from textlong.workshop import new_chat

state = {
    'root': {
        'content': None,
        'messages': [[{'role':'user', 'content':'hi'}, {'role':'assistant', 'content':'Can I help you?'}]]
    },
    'knowledge': [{
        'content': 'kg1',
        'messages': []
    }]
}
new_chat(state)
state

[{'messages': ['hi']}, {'messages': []}]

**1级新对话**

In [17]:
from textlong.workshop import new_chat

state = {
    'root': {
        'content': None,
        'messages': [[{'role':'user', 'content':'hi'}, {'role':'assistant', 'content':'Can I help you?'}]]
    },
    'knowledge': [{
        'content': 'kg1',
        'messages': []
    }],
    'output': [
        {
            'template_id': 'IDEA',
            'content': '# 这是一个好的开始',
            'messages': []
        },
        {
            'template_id': 'OUTLINE',
            'content': '# 这是一个好的开始\n<OUTLINE>请按我的要求扩写</OUTLINE>',
            'messages': []
        }
    ]
}

new_chat(state, -1)
state

[{'messages': ['hi']},
 {'messages': [], 'output': [{'output_markdown': '# 这是一个好的开始'}]}]

**2级新对话**

In [ ]:
from textlong.workshop import new_chat

state = [
    {
        'knowledge': [],
        'messages': [{'role':'user', 'content':'hi'}, {'role':'assistant', 'content':'Can I help you?'}]
    },
    {
        'knowledge': [],
        'output': [
            {
                'output_markdown': '# 这是一个好的开始',
                'messages': [{'role':'user', 'content':'hi'}, {'role':'assistant', 'content':'Can I help you?'}]
            },
            {
                'output_markdown': '# 这是一个好的开始',
                'outline': 
                'messages': [],
                'from_outline': {
                    '提纲1': '',
                    '提纲2': '',
                },
            }
        ]
    }
]
new_chat(state, -1, -1)
state

## 生成提纲

In [1]:
from textlong.llm import qwen
from textlong.workshop import write, chat, list_markdown
messages = []
state = {}

In [2]:
write(qwen, "OUTLINE", {"task": "帮我写一份markdown极简介绍"}, messages, state)

# Markdown 极简介绍

Markdown 是一种轻量级的文本格式语言，允许人们使用易读易写的纯文本格式编写文档，然后转换成结构化的HTML（超文本标记语言）文档。它被设计为“对人类友好”，同时能被轻松转换成有效的HTML代码，非常适合网页写作、记笔记、编写文档等。

## 一、Markdown 基础语法

### 1. 标题
<OUTLINE>
扩写要求：
- 介绍#号定义标题的规则，从`#`到`######`六级标题
- 预估字数：100字
</OUTLINE>

### 2. 段落与换行
<OUTLINE>
扩写要求：
- 解释段落的自然形成及两个空格或换行符实现换行的方法
- 预估字数：80字
</OUTLINE>

### 3. 强调与字体样式
<OUTLINE>
扩写要求：
- 说明使用`*`或`_`包裹文本实现斜体，双`**`或`__`实现粗体的规则
- 简述删除线(`~~`)与下划线(`<u>`)的非标准用法
- 预估字数：120字
</OUTLINE>

## 二、列表与代码

### 1. 无序列表
<OUTLINE>
扩写要求：
- 介绍使用`-`、`+`或`*`作为项目符号创建无序列表
- 预估字数：60字
</OUTLINE>

### 2. 有序列表
<OUTLINE>
扩写要求：
- 说明数字后跟`.`创建有序列表的规则
- 预估字数：40字
</OUTLINE>

### 3. 代码块与行内代码
<OUTLINE>
扩写要求：
- 描述使用三个反引号(```)包围代码块，以及使用反引号(``)包裹行内代码的用法
- 简要提及高亮语法的选项
- 预估字数：100字
</OUTLINE>

## 三、链接与图片

### 1. 链接
<OUTLINE>
扩写要求：
- 说明使用`[文本](URL)`格式创建链接的方法
- 简述参考链接的定义与使用(`[文本][id]`，`[id]: URL`)
- 预估字数：90字
</OUTLINE>

### 2. 图片
<OUTLINE>
扩写要求：
- 介绍插入图片的语法(`![alt文本](图片URL)`)，并解释`alt文本`的作用
- 预估字数：60字
</OUTLINE>

## 四、其他高级功能

### 1. 表格
<OUTLINE>
扩写要求：
- 简述使用管道符(`|`)和

'# Markdown 极简介绍\n\nMarkdown 是一种轻量级的文本格式语言，允许人们使用易读易写的纯文本格式编写文档，然后转换成结构化的HTML（超文本标记语言）文档。它被设计为“对人类友好”，同时能被轻松转换成有效的HTML代码，非常适合网页写作、记笔记、编写文档等。\n\n## 一、Markdown 基础语法\n\n### 1. 标题\n<OUTLINE>\n扩写要求：\n- 介绍#号定义标题的规则，从`#`到`######`六级标题\n- 预估字数：100字\n</OUTLINE>\n\n### 2. 段落与换行\n<OUTLINE>\n扩写要求：\n- 解释段落的自然形成及两个空格或换行符实现换行的方法\n- 预估字数：80字\n</OUTLINE>\n\n### 3. 强调与字体样式\n<OUTLINE>\n扩写要求：\n- 说明使用`*`或`_`包裹文本实现斜体，双`**`或`__`实现粗体的规则\n- 简述删除线(`~~`)与下划线(`<u>`)的非标准用法\n- 预估字数：120字\n</OUTLINE>\n\n## 二、列表与代码\n\n### 1. 无序列表\n<OUTLINE>\n扩写要求：\n- 介绍使用`-`、`+`或`*`作为项目符号创建无序列表\n- 预估字数：60字\n</OUTLINE>\n\n### 2. 有序列表\n<OUTLINE>\n扩写要求：\n- 说明数字后跟`.`创建有序列表的规则\n- 预估字数：40字\n</OUTLINE>\n\n### 3. 代码块与行内代码\n<OUTLINE>\n扩写要求：\n- 描述使用三个反引号(```)包围代码块，以及使用反引号(``)包裹行内代码的用法\n- 简要提及高亮语法的选项\n- 预估字数：100字\n</OUTLINE>\n\n## 三、链接与图片\n\n### 1. 链接\n<OUTLINE>\n扩写要求：\n- 说明使用`[文本](URL)`格式创建链接的方法\n- 简述参考链接的定义与使用(`[文本][id]`，`[id]: URL`)\n- 预估字数：90字\n</OUTLINE>\n\n### 2. 图片\n<OUTLINE>\n扩写要求：\n- 介绍插入图片的语法(`![alt文本](图片URL)`)，并解释`alt文本`的作用\n- 预估字数：60字

In [7]:
from textlong.workshop import list_markdown
list_markdown(state['outline_docs'].get_outlines())

[('OU-241-007',
  '<OUTLINE>\n扩写要求：\n- 介绍#号定义标题的规则，从`#`到`######`六级标题\n- 预估字数：100字\n</OUTLINE>\n\n'),
 ('OU-188-010',
  '<OUTLINE>\n扩写要求：\n- 解释段落的自然形成及两个空格或换行符实现换行的方法\n- 预估字数：80字\n</OUTLINE>\n\n'),
 ('OU-332-013',
  '<OUTLINE>\n扩写要求：\n- 说明使用`*`或`_`包裹文本实现斜体，双`**`或`__`实现粗体的规则\n- 简述删除线(`~~`)与下划线(`<u>`)的非标准用法\n- 预估字数：120字\n</OUTLINE>\n\n'),
 ('OU-744-018',
  '<OUTLINE>\n扩写要求：\n- 介绍使用`-`、`+`或`*`作为项目符号创建无序列表\n- 预估字数：60字\n</OUTLINE>\n\n'),
 ('OU-578-021',
  '<OUTLINE>\n扩写要求：\n- 说明数字后跟`.`创建有序列表的规则\n- 预估字数：40字\n</OUTLINE>\n\n'),
 ('OU-116-024',
  '<OUTLINE>\n扩写要求：\n- 描述使用三个反引号(```)包围代码块，以及使用反引号(``)包裹行内代码的用法\n- 简要提及高亮语法的选项\n- 预估字数：100字\n</OUTLINE>\n\n'),
 ('OU-378-029',
  '<OUTLINE>\n扩写要求：\n- 说明使用`[文本](URL)`格式创建链接的方法\n- 简述参考链接的定义与使用(`[文本][id]`，`[id]: URL`)\n- 预估字数：90字\n</OUTLINE>\n\n'),
 ('OU-524-032',
  '<OUTLINE>\n扩写要求：\n- 介绍插入图片的语法(`![alt文本](图片URL)`)，并解释`alt文本`的作用\n- 预估字数：60字\n</OUTLINE>\n\n'),
 ('OU-464-037',
  '<OUTLINE>\n扩写要求：\n- 简述使用管道符(`|`)和短横线(`-`)创建表格的格式\n- 提及对齐方式的设置\n- 预估字数：100字\n</OUT

## 从提纲扩写

In [8]:
state

{'outline_docs': <textlong.workshop.markdown.Markdown at 0x10b0eaa70>,
 'outline_task': [Document(page_content='<OUTLINE>\n扩写要求：\n- 介绍#号定义标题的规则，从`#`到`######`六级标题\n- 预估字数：100字\n</OUTLINE>\n\n', metadata={'id': '9984-241-007', 'type': 'OUTLINE'}),
  Document(page_content='<OUTLINE>\n扩写要求：\n- 解释段落的自然形成及两个空格或换行符实现换行的方法\n- 预估字数：80字\n</OUTLINE>\n\n', metadata={'id': '9984-188-010', 'type': 'OUTLINE'}),
  Document(page_content='<OUTLINE>\n扩写要求：\n- 说明使用`*`或`_`包裹文本实现斜体，双`**`或`__`实现粗体的规则\n- 简述删除线(`~~`)与下划线(`<u>`)的非标准用法\n- 预估字数：120字\n</OUTLINE>\n\n', metadata={'id': '9984-332-013', 'type': 'OUTLINE'}),
  Document(page_content='<OUTLINE>\n扩写要求：\n- 介绍使用`-`、`+`或`*`作为项目符号创建无序列表\n- 预估字数：60字\n</OUTLINE>\n\n', metadata={'id': '9984-744-018', 'type': 'OUTLINE'}),
  Document(page_content='<OUTLINE>\n扩写要求：\n- 说明数字后跟`.`创建有序列表的规则\n- 预估字数：40字\n</OUTLINE>\n\n', metadata={'id': '9984-578-021', 'type': 'OUTLINE'}),
  Document(page_content='<OUTLINE>\n扩写要求：\n- 描述使用三个反引号(```)包围代码块，以及使用反引号(``)包裹行内代码的用法\n- 简要提及高亮语法的

## 局部扩写

## 多轮对话 + 提纲扩写

## 加载数据

## 分析数据

## 多轮对话 + 分析数据

## 局部扩写 + 分析数据